# Hacker News Analysis

## Introduction

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

We're interested in posts with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just something interesting.

We will start by reading the `hacker_news.csv` file:

In [26]:
from csv import reader

with open('hacker_news.csv') as raw_file:
    raw_file_read = reader(raw_file)
    hn = list(raw_file_read)

# Separating header from content
hn_headers = hn[0]
hn = hn[1:]

# Display the table
print(hn_headers)
print('\n')
for row in hn[:5]:
    print(row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




## Extracting Ask HN and Show HN Posts

Let's separate posts beginning with `Ask HN` and `Show HN` (and case variations) into two different lists.

In [53]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(f'Ask posts  : {len(ask_posts)}')
print(f'Show posts : {len(show_posts)}')
print(f'Other posts: {len(other_posts)}')
print('------------------')
print(f'Total posts: {len(hn)}')

Ask posts  : 1744
Show posts : 1162
Other posts: 17194
------------------
Total posts: 20100


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Next, let's determine if ask posts or show posts receive more comments on average.

In [65]:
# Calculation for Ask comments
total_ask_comments = 0
for post in ask_posts:
    total_ask_comments += int(post[4])

avg_ask_comments = total_ask_comments / len(ask_posts)
print(f'Average number of comments for Ask HN: {int(avg_ask_comments)}')

# Calculation for Show comments
total_show_comments = 0
for post in show_posts:
    total_show_comments += int(post[4])

avg_show_comments = total_show_comments / len(show_posts)

print(f'Average number of comments for Show HN: {int(avg_show_comments)}')


Average number of comments for Ask HN: 14
Average number of comments for Show HN: 10


From these results we can see that `Ask HN` posts get more comments on average than `Show HN` posts.
Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. 

## Finding and Calculating the Number of Ask Posts and Comments by Hour Created

Next, we'll determine if ask posts created at a certain time are more likely to attract comments.

In [120]:
# Calculate the number of ask posts and comments by hour created
import datetime as dt

result_list = []
counts_by_hour = {}
comments_by_hour = {}

for post in ask_posts:
    result_list.append([post[6], int(post[4])])

for row in result_list:
    date_time_str = row[0]
    date_time = dt.datetime.strptime(date_time_str, '%m/%d/%Y %H:%M')
    hour = date_time.strftime('%H')

    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
# Calculate the average number of comments per post for posrts created during each hour
avg_by_hour = []

for hour in comments_by_hour:
    for hours in counts_by_hour:
        if hour == hours:
            avg_by_hour.append([hour, round(comments_by_hour[hour] / counts_by_hour[hours], 2)])

for row in avg_by_hour:
    print(row)

['09', 5.58]
['13', 14.74]
['10', 13.44]
['14', 13.23]
['16', 16.8]
['23', 7.99]
['12', 9.41]
['17', 11.46]
['15', 38.59]
['21', 16.01]
['20', 21.52]
['02', 23.81]
['18', 13.2]
['03', 7.8]
['05', 10.09]
['19', 10.8]
['01', 11.38]
['22', 6.75]
['08', 10.25]
['04', 7.17]
['00', 8.13]
['06', 9.02]
['07', 7.85]
['11', 11.05]


Let's now sort the list and print the five highest values:

In [149]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments\n")
for row in sorted_swap[:5]:
    hour_dt = dt.datetime.strptime(row[1], '%H')
    hour = hour_dt.strftime('%H:%M')
    print(f'{hour} : {row[0]} average comments per post')

Top 5 Hours for Ask Posts Comments

15:00 : 38.59 average comments per post
02:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.8 average comments per post
21:00 : 16.01 average comments per post


## Conclusion

In our analysis, we found that Ask HN posts receive more comments than Show HN posts. We further explored this and discovered that posts made at certain times, particularly around 15:00, are more likely to receive a higher number of comments.